In [1]:
# !pip install -U pip -q
# #!pip install accelerate==0.23.0 -q
# !pip install appdirs==1.4.4 -q
# !pip install bitsandbytes==0.41.0 -q
# #!pip install datasets==2.10.1 -q
# !pip install fire==0.5.0 -q
# #!pip install git+https://github.com/huggingface/peft.git -q
# #!pip install git+https://github.com/huggingface/transformers.git -q
# !pip install torch==2.5.1 -q
# #!pip install sentencepiece==0.1.97 -q
# !pip install tensorboardX==2.6 -q
# !pip install gradio==3.23.0 -q
!pip install faiss-cpu -q
!pip install faiss-gpu -q
!pip install deep-translator -q
!pip install langdetect -q
# !pip install triton -q
# !pip install unsloth-zoo -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `xuyxuypenis` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authentic

In [3]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [4]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import transformers
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
import faiss
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from nltk import sent_tokenize
import nltk
from deep_translator import GoogleTranslator
from langdetect import detect
import pickle
import transformers
from transformers import LlamaTokenizer, LlamaForCausalLM, BitsAndBytesConfig
from peft import (
    LoraConfig,
    get_peft_model
)
from sklearn.model_selection import train_test_split
from unsloth import FastLanguageModel
import json

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [9]:
!git lfs install

Git LFS initialized.


In [10]:
!git clone https://huggingface.co/bivba/quant

Cloning into 'quant'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 28 (delta 5), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (28/28), 16.54 KiB | 941.00 KiB/s, done.

Exiting because of "interrupt" signal.
You can inspect what was checked out with 'git status'
and retry with 'git restore --source=HEAD :/'

^C


In [15]:
token = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device='cuda')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

client = GoogleTranslator(source='ru', target='en')
text = open('/content/drive/MyDrive/quant/context.txt').read()

# tokenizer = LlamaTokenizer.from_pretrained(model_name)
# tokenizer.pad_token_id = 0
# tokenizer.padding_side = 'left'
#model = LlamaForCausalLM.from_pretrainde(model_name)

In [11]:
#model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit"

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Mistral-Nemo-Base-2407-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

==((====))==  Unsloth 2024.12.4: Fast Mistral patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
nltk.download('punkt_tab')
def split_document(document, chunk_size=500, overlap=50):
    sentences = sent_tokenize(document)
    chunks = []
    current_chunk = []
    current_length = 0

    for sentence in sentences:
        sentence_length = len(sentence)
        if current_length + sentence_length <= chunk_size:
            current_chunk.append(sentence)
            current_length += sentence_length
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = current_chunk[-overlap:] + [sentence]
            current_length = sum(len(s) for s in current_chunk)

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [16]:
chunks = split_document(text)

encoded = token.encode(chunks, return_numpy=True)

In [17]:
def add_index(encoded):
  num_vectors = len(encoded)
  dim = encoded.shape[1]
  faiss_index = faiss.IndexFlatL2(dim)

  faiss_index.add(np.array(encoded, dtype=np.float32))

In [18]:
faiss_index = faiss.read_index('/content/drive/MyDrive/quant/context (1).faiss')

In [19]:
def find_context(query, k=3):
  if detect(query) == 'ru':
    query = client.translate(query)
  q_emb = token.encode([query])

  _, indices = faiss_index.search(np.array(q_emb), k)
  retrieved_chunks = [chunks[i] for i in indices[0]]
  context = ' '.join(retrieved_chunks)
  return context

In [20]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.12.4 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


In [21]:
from datasets import load_dataset

In [25]:
mistral_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["question"]
    inputs       = [find_context(instruction) for instruction in instructions]
    outputs      = examples["answer"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = mistral_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

qas = load_dataset('json', data_files='/content/drive/MyDrive/quant/qa.json', split='train')
qas = qas.map(formatting_prompts_func, batched=True,)

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

In [32]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = qas,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        #max_steps = 60,
        num_train_epochs = 3, # For longer training runs!
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [30]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
12.916 GB of memory reserved.


In [33]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 90 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 33
 "-____-"     Number of trainable parameters = 114,032,640


Step,Training Loss
1,0.971800
2,0.901300
3,0.914500
4,0.918900
5,1.087200
6,0.896200
7,0.969500
8,1.041000
9,0.944600
10,1.005100


In [34]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    mistral_prompt.format(
        " Какое утверждение не является полстулатом квантовой теории \\\\\n1)Принцип неопределённости Гейзенберга \\\\\n2)Эволюция состояний \\\\\n3)Результатом измерения является число, принадлежащее спектру оператора \\\\\n4)вид оператора импульса \\\\\n", # instruction
        find_context(" Какое утверждение не является полстулатом квантовой теории \\\\\n1)Принцип неопределённости Гейзенберга \\\\\n2)Эволюция состояний \\\\\n3)Результатом измерения является число, принадлежащее спектру оператора \\\\\n4)вид оператора импульса \\\\\n"), # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True, pad_token_id = tokenizer.eos_token_id)
tokenizer.batch_decode(outputs)

['<s>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n Какое утверждение не является полстулатом квантовой теории \\\\\n1)Принцип неопределённости Гейзенберга \\\\\n2)Эволюция состояний \\\\\n3)Результатом измерения является число, принадлежащее спектру оператора \\\\\n4)вид оператора импульса \\\\\n\n\n### Input:\nIn this section, we will formulate a general quantum measurement framework based on basic principles of observable and measurement in quantum mechanics discussed in Chapters 1 and 2. #### Review of Basic Principles on Quantum Measurements\n\nThe Observable Postulate and the Measurement Postulate are two fundamental postulates in quantum mechanics that describe the behavior of quantum systems and the act of measuring those systems. The Observable Postulate states that every observable quantity in quantum mechanics is represented by a Hermitian oper

In [ ]:
model.push_to_hub("bivba/quant_4bit", token = "...") # Online saving
tokenizer.push_to_hub("bivba/quant_4bit", token = "...") # Online saving

In [ ]:
model.push_to_hub_merged("bivba/quant", tokenizer, save_method = "merged_16bit", token = "...")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 8.3G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 2.89 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 40/40 [07:24<00:00, 11.12s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00005.bin...
Unsloth: Saving model/pytorch_model-00002-of-00005.bin...
Unsloth: Saving model/pytorch_model-00003-of-00005.bin...
Unsloth: Saving model/pytorch_model-00004-of-00005.bin...
Unsloth: Saving model/pytorch_model-00005-of-00005.bin...
Done.


Unsloth: You are pushing to hub, but you passed your HF username = bivba.
We shall truncate bivba/quant to quant


...

pytorch_model-00005-of-00005.bin:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

pytorch_model-00001-of-00005.bin:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

pytorch_model-00003-of-00005.bin:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/bivba/quant
